In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading and Preprocessing data**

In [3]:
import pandas as pd
#from sklearn.preprocessing import StandardScaler
# Loading and preprocessing each dataset


def load_and_preprocess_data(file_path, dataset_name):
    # Loading the data
    data = pd.read_csv(file_path,encoding='latin1')




    return data




# Loading and preprocessing each dataset


demographic_data = load_and_preprocess_data(
    '/content/drive/MyDrive/demographic - demographic.csv',
    dataset_name='demographic'
)


diet_data = load_and_preprocess_data(
    '/content/drive/MyDrive/diet - diet.csv',
    dataset_name='diet'
)


examination_data = load_and_preprocess_data(
    '/content/drive/MyDrive/examination - examination.csv',
    dataset_name='examination'
)


labs_data = load_and_preprocess_data(
    '/content/drive/MyDrive/labs - labs.csv',
    dataset_name='labs'
)


medications_data = load_and_preprocess_data(
    '/content/drive/MyDrive/medications - medications.csv',
    dataset_name='medications'
)
labeled_data=load_and_preprocess_data(
    '/content/drive/MyDrive/Labeled_data - Sheet1 (1) - Labeled_data - Sheet1 (1).csv',
    dataset_name='labeled_data'
)




demographic_data = demographic_data.iloc[:, :4]
diet_data = diet_data.iloc[:, :5]
examination_data = examination_data.iloc[:, :6]
labs_data = labs_data.iloc[:, :5]
medications_data = medications_data.iloc[:, :3]

# Merging the datasets based on common column
data = pd.merge(demographic_data, diet_data,how='left')
data = pd.merge(data, examination_data,how='left')
data = pd.merge(data, labs_data,how='left')
data = pd.merge(data, medications_data,how='left')
data = pd.merge(data, labeled_data,how='left')
data=data.dropna()
print(data)


        SEQN  AGE  GENDER  COUNTRY  SUGARS    FATS  FIBERS  WATER_INTAKE  \
1      73557   69       1      1.0  176.47   52.81    10.8         960.0   
2      73558   54       1      1.0   44.99  124.29    16.7         360.0   
3      73558   54       1      1.0   44.99  124.29    16.7         360.0   
4      73558   54       1      1.0   44.99  124.29    16.7         360.0   
5      73558   54       1      1.0   44.99  124.29    16.7         360.0   
...      ...  ...     ...      ...     ...     ...     ...           ...   
20182  83724   80       1      1.0  164.56   69.26    22.2         690.0   
20183  83724   80       1      1.0  164.56   69.26    22.2         690.0   
20184  83724   80       1      1.0  164.56   69.26    22.2         690.0   
20185  83724   80       1      1.0  164.56   69.26    22.2         690.0   
20186  83724   80       1      1.0  164.56   69.26    22.2         690.0   

       BP_DIA  BP_SYS  HEIGHT  WEIGHT   BMI  CHOLESTROL  HAEMOGLOBIN  \
1        72.0  

**Prediction using CNN**

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



# Defining input features
input_features = ['AGE', 'GENDER','COUNTRY','SUGARS','FATS','FIBERS','WATER_INTAKE',
                  'BP_DIA','BP_SYS','HEIGHT','WEIGHT','BMI','CHOLESTROL','HAEMOGLOBIN',
                  'PLATELETS','URINE','DAYS','COUNT']

# Splitting the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Preparing input data and labels
train_features = train_data[input_features]
train_labels = train_data['SCORE']

test_features = test_data[input_features]
test_labels = test_data['SCORE']

# Converting NumPy arrays to tensors
train_features = tf.convert_to_tensor(train_features.values, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels.values, dtype=tf.float32)

test_features = tf.convert_to_tensor(test_features.values, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels.values, dtype=tf.float32)

# Defining the 1D CNN model
input_layer = Input(shape=(len(input_features), 1))  # Input shape for 1D CNN
conv1d_layer = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
maxpooling1d_layer = MaxPooling1D(pool_size=2)(conv1d_layer)
flatten_layer = Flatten()(maxpooling1d_layer)
dense_layer_1 = Dense(64, activation='relu')(flatten_layer)
output_layer = Dense(1, activation='linear')(dense_layer_1)  # Linear activation for regression task

model = Model(inputs=input_layer, outputs=output_layer)

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model
model.fit(train_features, train_labels, epochs=10, batch_size=32)

# Evaluating the model on test data
loss = model.evaluate(test_features, test_labels)
print(f"Test Loss (MSE): {loss}")

# Making predictions on new data
new_data = pd.DataFrame({
    'AGE': [69],
    'GENDER': [1],
    'COUNTRY': [1.0],
    'SUGARS': [176.47],
    'FATS': [52.81],
    'FIBERS': [10.8],
    'WATER_INTAKE': [960.0],
    'BP_DIA': [72.0],
    'BP_SYS': [122.0],
    'HEIGHT': [171.3],
    'WEIGHT':[78.3],
    'BMI':[26.7],
    'CHOLESTROL':[167.0],
    'HAEMOGLOBIN':[29.9],
    'PLATELETS':[204.0],
    'URINE':[0.821],
    'DAYS':[1460.0],
    'COUNT':[2.0]
})

# Converting the new data to a tensor
new_data = tf.convert_to_tensor(new_data.values, dtype=tf.float32)

# Reshaping the input data to match the 1D CNN input shape
new_data = tf.reshape(new_data, shape=(1, len(input_features), 1))

predicted_scores = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_scores[0][0]}")


Epoch 1/10
201/201 [==============================] - 2s 4ms/step - loss: 2736045.5000
Epoch 2/10
201/201 [==============================] - 1s 4ms/step - loss: 2218711.5000
Epoch 3/10
201/201 [==============================] - 0s 2ms/step - loss: 2299726.2500
Epoch 4/10
201/201 [==============================] - 0s 2ms/step - loss: 2169440.7500
Epoch 5/10
201/201 [==============================] - 0s 2ms/step - loss: 2155644.0000
Epoch 6/10
201/201 [==============================] - 0s 2ms/step - loss: 2188549.0000
Epoch 7/10
201/201 [==============================] - 0s 2ms/step - loss: 2147889.2500
Epoch 8/10
201/201 [==============================] - 0s 2ms/step - loss: 2149138.5000
Epoch 9/10
201/201 [==============================] - 0s 2ms/step - loss: 2142955.5000
Epoch 10/10
51/51 [==============================] - 0s 2ms/step - loss: 2360360.0000
Test Loss (MSE): 2360360.0
1/1 [==============================] - 0s 103ms/step
Predicted Healthcare Score: 2237.3076171875


**Prediction using feedforward network**

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Defining input features
input_features = ['AGE', 'GENDER','COUNTRY','SUGARS','FATS','FIBERS','WATER_INTAKE',
                  'BP_DIA','BP_SYS','HEIGHT','WEIGHT','BMI','CHOLESTROL','HAEMOGLOBIN',
                  'PLATELETS','URINE','DAYS','COUNT']

# Splitting the data into train and test sets

train_data, test_data = train_test_split(data, test_size=0.33, random_state=42)

# Preparing input data
train_features = train_data[input_features]
train_labels = train_data['SCORE']

train_features = tf.convert_to_tensor(train_features.values, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels.values, dtype=tf.float32)

# Defining the neural network model
input_layer = Input(shape=(len(input_features),))
dense_layer_1 = Dense(64, activation='relu')(input_layer)
dense_layer_2 = Dense(32, activation='linear')(dense_layer_1)
output_layer = Dense(1, activation='linear')(dense_layer_2)  # Linear activation for regression task

model = Model(inputs=input_layer, outputs=output_layer)

# Compiling the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Training the model
model.fit(train_features, train_labels, epochs=10, batch_size=32)

# Evaluating the model on test data
test_features = test_data[input_features]
test_labels = test_data['SCORE']

test_features = tf.convert_to_tensor(test_features.values, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels.values, dtype=tf.float32)

loss = model.evaluate(test_features, test_labels)
print(f"Test Loss: {loss}")

# Making predictions on new data
new_data = pd.DataFrame({
    'AGE': [69],
    'GENDER': [1],
    'COUNTRY': [1.0],
    'SUGARS': [176.47],
    'FATS': [52.81],
    'FIBERS': [10.8],
    'WATER_INTAKE': [960.0],
    'BP_DIA': [72.0],
    'BP_SYS': [122.0],
    'HEIGHT': [171.3],
    'WEIGHT':[78.3],
    'BMI':[26.7],
    'CHOLESTROL':[167.0],
    'HAEMOGLOBIN':[29.9],
    'PLATELETS':[204.0],
    'URINE':[0.821],
    'DAYS':[1460.0],
    'COUNT':[2.0]
})
new_data = tf.convert_to_tensor(new_data.values, dtype=tf.float32)

predicted_scores = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_scores[0][0]}")


Epoch 1/10
168/168 [==============================] - 1s 3ms/step - loss: 1361.4456
Epoch 2/10
168/168 [==============================] - 0s 3ms/step - loss: 1099.7804
Epoch 3/10
168/168 [==============================] - 0s 3ms/step - loss: 1089.5010
Epoch 4/10
168/168 [==============================] - 0s 3ms/step - loss: 1080.6005
Epoch 5/10
168/168 [==============================] - 1s 4ms/step - loss: 1099.0491
Epoch 6/10
168/168 [==============================] - 1s 4ms/step - loss: 1087.3910
Epoch 7/10
168/168 [==============================] - 1s 4ms/step - loss: 1091.8584
Epoch 8/10
168/168 [==============================] - 1s 4ms/step - loss: 1086.7251
Epoch 9/10
168/168 [==============================] - 1s 4ms/step - loss: 1086.4760
Epoch 10/10
83/83 [==============================] - 0s 3ms/step - loss: 1122.8374
Test Loss: 1122.83740234375
1/1 [==============================] - 0s 177ms/step
Predicted Healthcare Score: 1827.273681640625


**Prediction using xgboost**

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb


# Defining input features
input_features = ['AGE', 'GENDER','COUNTRY','SUGARS','FATS','FIBERS','WATER_INTAKE',
                  'BP_DIA','BP_SYS','HEIGHT','WEIGHT','BMI','CHOLESTROL','HAEMOGLOBIN',
                  'PLATELETS','URINE','DAYS','COUNT']

# Splitting the data into train and test sets
X = data[input_features].values
y = data['SCORE'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Defining the XGBoost-based recommendation model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5, learning_rate=0.1)

# Training the model
model.fit(X_train, y_train)

# Evaluating the model on test data
y_pred = model.predict(X_test)
mse = np.mean((y_test - y_pred) ** 2)
print(f"Test Mean Squared Error (MSE): {mse}")

# Making predictions on new data
new_data = pd.DataFrame({
    'AGE': [69],
    'GENDER': [1],
    'COUNTRY': [1.0],
    'SUGARS': [176.47],
    'FATS': [52.81],
    'FIBERS': [10.8],
    'WATER_INTAKE': [960.0],
    'BP_DIA': [72.0],
    'BP_SYS': [122.0],
    'HEIGHT': [171.3],
    'WEIGHT':[78.3],
    'BMI':[26.7],
    'CHOLESTROL':[167.0],
    'HAEMOGLOBIN':[29.9],
    'PLATELETS':[204.0],
    'URINE':[0.821],
    'DAYS':[1460.0],
    'COUNT':[2.0]
})



predicted_score = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_score[0]}")


Test Mean Squared Error (MSE): 879933.7129817194
Predicted Healthcare Score: 2081.90625


**Prediction using random forests**

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Defining input features
input_features = ['AGE', 'GENDER','COUNTRY','SUGARS','FATS','FIBERS','WATER_INTAKE',
                  'BP_DIA','BP_SYS','HEIGHT','WEIGHT','BMI','CHOLESTROL','HAEMOGLOBIN',
                  'PLATELETS','URINE','DAYS','COUNT']

# Splitting the data into train and test sets
X = data[input_features].values
y = data['SCORE'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the input data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Defining the Random Forest-based recommendation model
model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Training the model
model.fit(X_train, y_train)

# Evaluating the model on test data
y_pred = model.predict(X_test)
mse = np.mean((y_test - y_pred) ** 2)
print(f"Test Mean Squared Error (MSE): {mse}")

# Making predictions on new data
new_data = pd.DataFrame({
    'AGE': [69],
    'GENDER': [1],
    'COUNTRY': [1.0],
    'SUGARS': [176.47],
    'FATS': [52.81],
    'FIBERS': [10.8],
    'WATER_INTAKE': [960.0],
    'BP_DIA': [72.0],
    'BP_SYS': [122.0],
    'HEIGHT': [171.3],
    'WEIGHT':[78.3],
    'BMI':[26.7],
    'CHOLESTROL':[167.0],
    'HAEMOGLOBIN':[29.9],
    'PLATELETS':[204.0],
    'URINE':[0.821],
    'DAYS':[1460.0],
    'COUNT':[2.0]
})
new_data = scaler.transform(new_data)

predicted_score = model.predict(new_data)
print(f"Predicted Healthcare Score: {predicted_score[0]}")


Test Mean Squared Error (MSE): 1849543.428304031
Predicted Healthcare Score: 2250.5584871243004


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
